In [6]:
from powerloss_functions import load_xgb_models, predict_power_and_voc
import numpy as np

### Define example latent vectors:
How to get these for your cell EL images is shown in other examples

In [7]:
example_latents = np.array([
    np.array([ 0.24241999,  0.45217514,  0.97815484,  1.4499949 , -1.5249883 ,
              -2.939314  ,  1.0984266 , -0.12293977, -1.1299406 ,  0.2869924 ,
               1.9682227 ,  1.0318435 ,  1.87186   , -0.8736567 ,  2.43369   ,
              -1.1833816 ,  1.549619  , -0.7797383 ,  0.96275795, -1.8737011 ,
              -0.08402814,  0.91361403,  1.0060663 ,  0.6487342 , -0.04528153,
               0.44201964, -1.223722  , -0.06355916,  0.65274584,  0.04935604,
              -0.27834928, -1.0977445 , -1.2030623 , -0.3235336 ,  0.2602179 ,
              -0.28784323,  1.1689687 ,  0.4854142 , -1.0354393 , -1.3085537 ,
              -0.6724297 ,  0.6303619 , -0.13833919, -0.5086113 , -0.55449027,
               0.96066123, -0.10699381,  0.46864828,  1.1066953 ,  1.9992326 ],
             dtype=np.float32),

    np.array([ 1.1702411 , -1.2032634 , -1.1532567 ,  1.283272  ,  1.4820137 ,
              -1.684932  ,  0.5730577 , -1.4129782 , -0.04049593,  0.38759586,
               0.93989795,  0.32529882, -0.2901097 ,  0.4274361 ,  0.09410781,
               0.29883933,  1.4859366 ,  1.5629989 , -0.16382706, -1.0706595 ,
              -0.23171969, -1.008157  , -0.2675244 , -1.1766199 ,  0.9864845 ,
               1.1019561 , -0.29842865,  0.51003385, -0.8262293 ,  0.5515144 ,
               0.7081789 ,  1.9557799 ,  0.669111  , -0.35770333,  0.09954366,
               0.38981187,  0.30818567,  1.1630924 ,  0.9415076 ,  0.7496076 ,
               1.503298  , -0.26176888,  0.00537223, -1.4165225 ,  0.8113852 ,
               0.24869177,  1.3110806 , -1.8022932 ,  0.59169745,  0.32176077],
             dtype=np.float32),

    np.array([-1.2923387 ,  0.8474884 ,  2.662438  ,  0.7604594 , -0.14168845,
              -3.0224955 , -1.4385438 ,  0.4840144 , -0.35280943, -0.13301083,
               0.6721701 ,  0.5446935 ,  0.8052741 , -1.9656703 ,  0.37340263,
               1.1418803 , -3.0987735 , -2.4352448 ,  0.12710397, -0.77480114,
               0.17418909, -0.45899892, -0.37250227, -1.1538354 , -0.27806127,
               0.33386093, -0.4410423 , -1.402402  ,  0.09539375,  0.08041148,
              -1.2225368 ,  1.1416847 ,  1.1382179 ,  0.15572737,  0.39324802,
               0.3373769 ,  0.16119203, -0.87466097,  0.05443547, -0.2696092 ,
               1.6436559 ,  0.7371242 , -0.29250032, -0.7745946 ,  1.1105676 ,
               0.01021414, -0.44243583,  2.5693357 , -1.2217515 ,  0.17328888],
             dtype=np.float32),

    np.array([ 0.21270931, -0.47887728, -1.3520491 ,  1.2311001 , -0.49645793,
              -0.215451  , -0.6477202 , -0.2579493 , -1.2793546 ,  1.557384  ,
               0.8669748 ,  1.4398419 , -0.05749279,  0.09622769,  1.9737855 ,
              -0.25288105,  1.4334834 , -1.1325804 ,  0.36544695, -0.03118681,
              -0.9547925 ,  0.80866206,  0.05026607,  1.7798712 , -1.3284695 ,
               0.29892215, -0.646554  , -0.42096144,  0.08981171, -0.3981455 ,
              -0.1211867 , -1.1044581 , -0.18419716, -0.01123017,  0.16615252,
              -0.70454276,  0.45732343,  1.387109  , -1.2196114 , -1.7286274 ,
              -0.1883303 , -0.74527794,  0.00890233,  0.33905134, -0.10815011,
               1.5033964 ,  0.04647948,  1.2081712 ,  0.61395484,  1.4679968 ],
             dtype=np.float32),

    np.array([-0.74178386, -0.28830415, -0.08164142, -0.2438497 , -0.23312329,
              -0.2891588 , -0.3581485 ,  0.66993487, -1.4835395 , -0.42535332,
               1.5485582 , -0.59646815,  0.8465934 , -0.44712684,  0.08246939,
               0.10648207,  0.3146866 , -1.0368568 ,  0.55821025, -0.47742945,
               1.405572  ,  0.32312307,  1.0303203 , -1.8350694 , -1.6701893 ,
              -0.67007583, -0.11833465, -0.1277042 ,  0.43137333,  0.607671  ,
              -1.3010788 ,  1.2269626 , -1.3319882 ,  0.43101764,  0.35383585,
               0.39769188,  0.09201814,  2.4513376 , -0.927063  ,  0.25916797,
               0.37646204, -0.9077203 ,  0.1464294 ,  0.37251297,  0.01247774,
               0.7865578 ,  0.08395594,  0.6235267 ,  0.70956576,  1.1011281 ],
             dtype=np.float32),
], dtype=object)

### Load xgboost models:

In [8]:
pmpp_model, voc_model = load_xgb_models(
        pmpp_model_path="xgb_model_pmpp_diff_percent_3CH.pkl",
        voc_model_path="xgb_model_Voc_diff_percent_3CH.pkl"
    )

### Predict delta Pmpp and Voc in %

In [9]:
df_predictions = predict_power_and_voc(example_latents, pmpp_model, voc_model)


In [10]:
df_predictions

,power_loss_%,Voc_diff_%
0,-14.947733,-14.874655
1,-23.281399,-17.710461
2,-16.396479,-18.207232
3,-8.646928,-13.734867
4,-8.522730,-19.516245
